In [ ]:
%run ./Obter_Dados_Com_Embeddings.ipynb

In [115]:
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

In [ ]:
setTreino, setTeste, targetTreino, targetTeste = train_test_split(datasetEmb, target, test_size=0.30, random_state=0)
setTreino.shape, setTeste.shape, targetTreino.shape, targetTeste.shape

In [117]:
def LimitarEntreZeroUm(treino, teste):
    treinoVal = []
    testeVal = []
    scaler = MinMaxScaler()
    
    for i in range(treino.shape[0]):
        treinoVal.append(scaler.fit_transform(treino[i]))
        
    for i in range(teste.shape[0]):
        testeVal.append(scaler.transform(teste[i]))
    
    return np.array(treinoVal), np.array(testeVal)

In [ ]:
setTreino, setTeste = setTreino.reshape(-1, 163, 203), setTeste.reshape(-1, 163, 203)
targetTreino, targetTeste = targetTreino.reshape(-1, 163), targetTeste.reshape(-1, 163)

setTreino.shape, setTeste.shape, targetTreino.shape, targetTeste.shape

In [119]:
multinomialNB = MultinomialNB()
bernoulliNB = BernoulliNB()
perceptron = Perceptron(random_state=0, max_iter=30)
passiveAggressive = PassiveAggressiveClassifier(max_iter=1000, random_state=0)
MLP = MLPClassifier(random_state=0, max_iter=300)
sgdc = SGDClassifier(random_state=0)

In [ ]:
models = [
    ('MultinomialNB', multinomialNB),
    ('BernoulliNB', bernoulliNB),
    ('Perceptron', perceptron),
    ('Passive Aggressive Classifier', passiveAggressive),
    ('MLPClassifier', MLP),
    ('SGDClassifier', sgdc)
]

dataFrames = []
results = []
nomes = []
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']

for nome, model in models:
    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=0)
    
    if nome == 'MultinomialNB':
        setTreinoVal, setTesteVal = LimitarEntreZeroUm(setTreino, setTeste)
    else:
        setTreinoVal, setTesteVal = setTreino, setTeste
    
    cv_results = model_selection.cross_validate(model, setTreinoVal.reshape(-1, 203), targetTreino.reshape(-1), cv=kfold, scoring=scoring)
    
    epochs = 10

    for k in range(epochs):
        for i in range(setTreinoVal.shape[0]):
            setBatch, targetBatch = setTreinoVal[i], targetTreino[i]
            model.partial_fit(setBatch, targetBatch, classes=classes)
            
    predicoes = []
    for j in range(setTesteVal.shape[0]):
        predicao = model.predict(setTesteVal[j])
        predicoes.extend(predicao.tolist())
    
    print(nome)
    print(classification_report(targetTeste.reshape(-1), predicoes, target_names=classes))
    
    results.append(cv_results)
    nomes.append(nome)
    
    dataFrameResultado = pd.DataFrame(cv_results)
    dataFrameResultado['model'] = nome
    dataFrames.append(dataFrameResultado)
    
resultadosFinais = pd.concat(dataFrames, ignore_index=True)

print(resultadosFinais)